# Advanced Workflows with Modular API

This notebook demonstrates advanced rheological analysis workflows using rheo's modular API for fine-grained control.

Topics covered:
1. Direct model instantiation from ModelRegistry
2. Custom optimization with constraints
3. Multi-start optimization to avoid local minima
4. Parameter sensitivity analysis

## Setup and Imports

In [ ]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from rheo.core.registry import ModelRegistry
from rheo.core.data import RheoData
from rheo.utils.optimization import nlsq_optimize

np.random.seed(42)

## Generate Realistic Polymer Relaxation Data

We'll simulate data from a fractional Maxwell model (gel variant) which exhibits power-law relaxation.

In [ ]:
# Generate time vector
time = np.logspace(-3, 2, 60)  # 0.001 to 100 seconds

# True parameters for fractional Maxwell gel model
c_alpha_true = 5e5  # Quasi-property (Pa·s^alpha)
alpha_true = 0.4    # Fractional order
eta_true = 1e4      # Viscosity (Pa·s)

# Generate synthetic data using Mittag-Leffler function
# G(t) = c_alpha * t^(-alpha) * E_{alpha}(-t^alpha / tau_alpha)
# Simplified approximation for demonstration
tau_alpha = eta_true / c_alpha_true
stress_true = c_alpha_true * time**(-alpha_true) * np.exp(-(time / tau_alpha)**(alpha_true))

# Add heteroscedastic noise (higher relative error at longer times)
noise_level = 0.03 + 0.02 * (time / time.max())
stress_noisy = stress_true * (1 + noise_level * np.random.randn(len(stress_true)))

# Create RheoData object
data = RheoData(
    x=time,
    y=stress_noisy,
    x_units='s',
    y_units='Pa',
    domain='time',
    test_mode='relaxation'
)

print(f"Generated {len(time)} data points")
print(f"Time range: {time[0]:.3f} to {time[-1]:.1f} seconds")
print(f"Stress range: {stress_noisy.min():.2e} to {stress_noisy.max():.2e} Pa")

In [ ]:
# Visualize the data
plt.figure(figsize=(9, 5))
plt.loglog(time, stress_noisy, 'o', label='Synthetic Data', alpha=0.6, markersize=5)
plt.loglog(time, stress_true, '--', label='True Model', color='red', linewidth=2)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Stress Modulus G(t) (Pa)', fontsize=12)
plt.title('Fractional Viscoelastic Relaxation Data', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Direct Model Instantiation from ModelRegistry

Instead of using the Pipeline API, we can directly create model instances for maximum control.

In [ ]:
# Create model instance directly
model = ModelRegistry.create('fractional_maxwell_gel')

print("\nModel Information:")
print(f"  Name: {model.__class__.__name__}")
print(f"  Number of parameters: {len(model.parameters)}")
print(f"\nDefault Parameters:")
for name, param in model.parameters.items():
    print(f"  {name}: {param.value:.3e} (bounds: {param.bounds})")

## Set Intelligent Initial Guesses

Good initial guesses can significantly improve optimization convergence.

In [ ]:
# Estimate initial values from data characteristics
G_initial = stress_noisy[0]  # Initial modulus
G_final = stress_noisy[-1]    # Final modulus
time_mid = time[len(time)//2]

# Set initial parameter guesses
model.parameters['c_alpha'].value = G_initial * 2  # Higher than initial stress
model.parameters['alpha'].value = 0.5               # Middle of typical range
model.parameters['eta'].value = 1e4                 # Typical viscosity scale

print("\nInitial Parameter Guesses:")
for name, param in model.parameters.items():
    print(f"  {name}: {param.value:.3e}")

## Custom Optimization with Constraints

We'll define a custom objective function and add physical constraints.

In [ ]:
def objective_function(params_array):
    """
    Objective function for optimization.
    Returns sum of squared residuals (weighted by inverse variance).
    """
    # Update model parameters
    param_names = list(model.parameters.keys())
    for i, name in enumerate(param_names):
        model.parameters[name].value = params_array[i]
    
    # Get model predictions
    predictions = model.predict(data)
    
    # Calculate weighted residuals (inverse variance weighting)
    weights = 1.0 / (noise_level * stress_noisy)**2
    residuals = (predictions - stress_noisy) * np.sqrt(weights)
    
    return np.sum(residuals**2)

# Get initial parameter values and bounds
initial_params = np.array([p.value for p in model.parameters.values()])
bounds = [p.bounds for p in model.parameters.values()]

print(f"\nOptimization Setup:")
print(f"  Number of parameters: {len(initial_params)}")
print(f"  Initial objective value: {objective_function(initial_params):.2e}")

### Add Physical Constraints

We can add constraints to ensure physically meaningful results.

In [ ]:
# Define constraints
constraints = []

# Constraint 1: alpha must be between 0 and 1
# (This is already handled by parameter bounds)

# Constraint 2: c_alpha should be reasonable relative to data scale
def constraint_c_alpha(params):
    c_alpha = params[0]
    return c_alpha - stress_noisy.min() / 10  # c_alpha > G_min/10

constraints.append({'type': 'ineq', 'fun': constraint_c_alpha})

print(f"Added {len(constraints)} custom constraints")

### Run Optimization

In [ ]:
# Optimize using scipy.optimize.minimize
result = minimize(
    objective_function,
    initial_params,
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 1000, 'ftol': 1e-9}
)

print("\n" + "="*60)
print("OPTIMIZATION RESULTS")
print("="*60)
print(f"Success: {result.success}")
print(f"Message: {result.message}")
print(f"Iterations: {result.nit}")
print(f"Final objective value: {result.fun:.3e}")
print(f"\nOptimized Parameters:")
param_names = list(model.parameters.keys())
for i, name in enumerate(param_names):
    true_val = [c_alpha_true, alpha_true, eta_true][i]
    print(f"  {name}: {result.x[i]:.3e} (true: {true_val:.3e})")

### Visualize Fit

In [ ]:
# Update model with optimized parameters
for i, name in enumerate(param_names):
    model.parameters[name].value = result.x[i]

# Get optimized predictions
predictions_opt = model.predict(data)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Data and fit
ax1.loglog(time, stress_noisy, 'o', label='Data', alpha=0.6, markersize=5)
ax1.loglog(time, stress_true, '--', label='True Model', color='red', linewidth=2)
ax1.loglog(time, predictions_opt, '-', label='Optimized Fit', color='green', linewidth=2)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Stress Modulus G(t) (Pa)', fontsize=12)
ax1.set_title('Optimized Fit', fontsize=13)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Residuals
residuals = (stress_noisy - predictions_opt) / stress_noisy * 100
ax2.semilogx(time, residuals, 'o-', alpha=0.6)
ax2.axhline(y=0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Fit Residuals', fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Multi-Start Optimization

To avoid local minima, we can try multiple initial guesses and select the best result.

In [ ]:
# Define multiple initial guesses using Latin Hypercube Sampling
n_starts = 5

# Generate initial guesses within parameter bounds
initial_guesses = []
for i in range(n_starts):
    guess = []
    for bound in bounds:
        # Sample logarithmically within bounds
        log_min, log_max = np.log10(bound[0]), np.log10(bound[1])
        log_val = log_min + (log_max - log_min) * np.random.rand()
        guess.append(10**log_val)
    initial_guesses.append(np.array(guess))

print(f"\nMulti-Start Optimization with {n_starts} initial guesses:")
print("="*60)

results_multi = []
for i, guess in enumerate(initial_guesses):
    result_i = minimize(
        objective_function,
        guess,
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 500}
    )
    results_multi.append(result_i)
    print(f"Start {i+1}: Objective = {result_i.fun:.3e}, Success = {result_i.success}")

# Select best result
best_idx = np.argmin([r.fun for r in results_multi])
best_result = results_multi[best_idx]

print(f"\nBest result from start {best_idx+1}:")
print(f"  Objective value: {best_result.fun:.3e}")
print(f"  Parameters:")
for i, name in enumerate(param_names):
    print(f"    {name}: {best_result.x[i]:.3e}")

## Parameter Sensitivity Analysis

Understand how sensitive the model predictions are to each parameter.

In [ ]:
# Use best-fit parameters as baseline
baseline_params = best_result.x.copy()

# Vary each parameter by ±20% and observe effect
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (name, ax) in enumerate(zip(param_names, axes)):
    # Create parameter variations
    variations = np.linspace(0.5, 1.5, 5)  # 50% to 150% of baseline
    
    for var in variations:
        params_varied = baseline_params.copy()
        params_varied[i] *= var
        
        # Update model and predict
        for j, pname in enumerate(param_names):
            model.parameters[pname].value = params_varied[j]
        
        pred = model.predict(data)
        
        label = f"{var*100:.0f}%" if var in [0.5, 1.0, 1.5] else None
        alpha = 1.0 if var == 1.0 else 0.4
        linewidth = 2.5 if var == 1.0 else 1.5
        
        ax.loglog(time, pred, alpha=alpha, linewidth=linewidth, label=label)
    
    ax.loglog(time, stress_noisy, 'o', color='black', alpha=0.3, markersize=3, label='Data')
    ax.set_xlabel('Time (s)', fontsize=10)
    ax.set_ylabel('G(t) (Pa)', fontsize=10)
    ax.set_title(f'Sensitivity to {name}', fontsize=11)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we explored advanced workflows using rheo's modular API:

### Key Techniques:

1. **Direct Model Instantiation**: Using `ModelRegistry.create()` for fine-grained control
2. **Custom Optimization**: Defining objective functions with weighted residuals and constraints
3. **Multi-Start Optimization**: Avoiding local minima by trying multiple initial guesses
4. **Sensitivity Analysis**: Understanding parameter influence on model predictions

### When to Use Modular API:

- Custom optimization strategies (constraints, multi-objective, Bayesian)
- Advanced parameter analysis (confidence intervals, correlations)
- Research applications requiring maximum flexibility
- Integration with other optimization frameworks

### When to Use Pipeline API:

- Standard workflows and quick analysis
- Batch processing multiple datasets
- Production/routine analysis
- Users new to rheological analysis

### Next Steps:

- Explore multi-technique fitting (see `multi_technique_fitting.ipynb`)
- Try model comparison with information criteria (see `multi_model_comparison.ipynb`)
- Apply Bayesian inference for uncertainty quantification (Phase 3 feature)